In [8]:
from brian2 import *

In [9]:
####################################################################################################
# Simulation parameters
dt = 1e-4                    # Timestep
t = np.arange(0,200000)*dt    # Simulation time array t
num_receptors = 101          # Number of receptor neurons
speed = 80                   # Speed of bright spot, number of receptor neurons passed per second
####################################################################################################

plot_t = []                  # Time of spikes
plot_n = []                  # Receptor neuron id's
currtime = 0                 # Time when a spike is fired
neuronid = 0                 # Neuron that fires a spike
stepdir = 1                  # Is the bright spot moving towards the right (1) or left (-1)
timestep = 1.0/(speed-1)     # Time between spikes generated by nearby receptor neurons

# Make placeholders for spike time arrays to be used for SNN simulation
spikes = []
for k in range(num_receptors):
    spikes.append([])

# Generate and plot spikes from receptor neurons
while currtime < t[-1]:
    plot_t.append(currtime)
    plot_n.append(neuronid)
    spikes[neuronid].append(currtime)
    if neuronid == 0 and stepdir == -1:
        stepdir = 1
    elif neuronid == (num_receptors-1) and stepdir == 1:
        stepdir = -1
    neuronid += stepdir
    currtime += timestep



In [10]:
tau = 20*ms
tau_syn = 2*ms
I_weight = 3*nA
threshold='v > -55*mV'
v_reset='v = -80*mV'
v_rest = -70*mV
R = 100*Mohm
equ = '''
dv/dt = -(v - v_rest)/tau + R*I_syn/tau : volt
dI_syn/dt = -I_syn/tau_syn : ampere
    '''

def initNetwork(spikeArr=[], timeArr=[]*second, inputSize = num_receptors):
    start_scope() 

    spikeArr = plot_n
    timeArr = plot_t*second

    # inputSize = num_receptors

    generator = SpikeGeneratorGroup(inputSize, spikeArr, timeArr)

    # generatorMonitor = SpikeMonitor(generator)
    
    relay = NeuronGroup(inputSize-1, equ, threshold=threshold, reset=v_reset, method=exact)
    relay.v = -70*mV
    inpToRel = Synapses(generator, relay, on_pre='I_syn += I_weight')
    inpToRel.connect(condition='j+1 == i')

    relay1 = NeuronGroup(inputSize-1, equ, threshold=threshold, reset=v_reset, method=exact)
    relay1.v = -70*mV
    inpToRel1 = Synapses(generator, relay1, on_pre='I_syn += I_weight')
    inpToRel1.connect(condition='j == i')

    inhib = NeuronGroup(inputSize, equ, threshold=threshold, reset=v_reset, method=exact)
    inhib.v = -70*mV
    inpToIn = Synapses(generator, inhib, on_pre='I_syn += I_weight')
    inpToIn.connect(condition='j == i')

    inToRelay = Synapses(inhib, relay, on_pre='I_syn -= I_weight')
    inToRelay.connect(condition='j == i')

    inToRelay1 = Synapses(inhib, relay1, on_pre='I_syn -= I_weight')
    inToRelay1.connect(condition='j+1 == i')

    output = NeuronGroup(2, equ, threshold=threshold, reset=v_reset, method=exact)
    output.v = -70*mV
    relayToOut = Synapses(relay, output, on_pre='I_syn += I_weight/3')
    relayToOut.connect(condition='j == 0')

    relay1ToOut = Synapses(relay1, output, on_pre='I_syn += I_weight/3')
    relay1ToOut.connect(condition='j == 1')

    # outputMonitor = SpikeMonitor(output)
    network = Network(generator, relay, inpToRel, relay1, inpToRel1, inhib, inpToIn, inToRelay, inToRelay1, output, relayToOut, relay1ToOut)
    return (network, output, generator)
    
    

In [11]:
(network, output, generator) = initNetwork(inputSize = 101)
network.store("init")

In [12]:
def runNet(network, output, generator, spikeArr, timeArr, runTime = 1000*ms):
    network.restore("init")
    generator.set_spikes(spikeArr, timeArr)
    outputMonitor = SpikeMonitor(output)
    network.add(outputMonitor)
    network.run(runTime)

    rightSpikes = 0
    leftSpikes = 0
    for s in outputMonitor.i:
        if s == 0:
            rightSpikes +=1
        else:
            leftSpikes +=1
    network.remove(outputMonitor)
    # print("Right direction:", rightSpikes, "Left directions:", leftSpikes)
    if(rightSpikes > leftSpikes):
        print(f'Right to left! (Right spikes :{str(rightSpikes)}, Left spikes :{str(leftSpikes)})')
    elif (rightSpikes < leftSpikes):
        print(f'Left to right! (Right spikes :{str(rightSpikes)}, Left spikes :{str(leftSpikes)})')
    else:
        print(f'Undecided! (Right spikes :{str(rightSpikes)}, Left spikes :{str(leftSpikes)})')

In [13]:
# spikeArr = [4,3,2,1,0] #Right to left
# timeArr = [10,20,30,40,50]*ms
# # spikeArr = [0,1,2,3,4] #Left to right
# # timeArr = [100,120,130,140,150]*ms

# spikeArr = plot_n
# timeArr = plot_t*second

# runNet(network, output, generator, spikeArr, timeArr)

In [14]:
import cv2
import numpy as np
import os
import time


def ResizeAndDisplayFrame(win_name, frame, size):
    resized = cv2.resize(frame, size)
    cv2.imshow(win_name, resized)
    return resized


frames_dir = "frames"
if not os.path.isdir(frames_dir):
    os.makedirs(frames_dir)

# Define the codec and create a VideoWriter object
codec = cv2.VideoWriter_fourcc(*'XVID')
fps = 25
frame_size_org = (960, 540)
output_file_org = 'output_org.avi'

out_org = cv2.VideoWriter(output_file_org, codec, fps, frame_size_org)

bin_th = 5
num_neurons = 101

vid = cv2.VideoCapture(0)

frame_count = 0
previous_frame = None

recording = False
start_time = 0
spikeArr = []
timeArr = []

while (True):

    frame_count += 1

    # Capture the video frame by frame
    ret, frame = vid.read()
    org_h, org_w = frame.shape[:2]

    # Resize to half so that the main frame does not occupy the entire screen
    # and flip it to create a mirror effect.
    org_w = int(org_w / 2)
    org_h = int(org_h / 2)
    frame = cv2.resize(frame, (org_w, org_h))
    frame = cv2.flip(frame, 1)
    cv2.imshow("Main Image", frame)

    if ((frame_count % 2) == 0):

        out_org.write(frame)
        # Resize the width to the number of neurons
        frame = cv2.resize(frame, (num_neurons, org_h), cv2.INTER_CUBIC)
        ResizeAndDisplayFrame("Resized", frame, (org_w, org_h))

        # Prepare image; grayscale and blur
        prepared_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        prepared_frame = cv2.GaussianBlur(src=prepared_frame,
                                          ksize=(0, 0),
                                          sigmaX=3)
        ResizeAndDisplayFrame("Blurred", prepared_frame, (org_w, org_h))

        # Set previous frame and continue if there is None
        if (previous_frame is None):
            # First frame; there is no previous one yet
            previous_frame = prepared_frame
            continue

        # Calculate difference and update previous frame
        diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
        ResizeAndDisplayFrame("Diff Frame", diff_frame, (org_w, org_h))

        previous_frame = prepared_frame

        # Sum the columns of the matrix
        sum_cols = cv2.reduce(diff_frame, 0, cv2.REDUCE_AVG)
        ResizeAndDisplayFrame("1D Diff", sum_cols, (org_w, 100))

        thresh_frame = cv2.threshold(src=sum_cols,
                                     thresh=bin_th,
                                     maxval=255,
                                     type=cv2.THRESH_BINARY)[1]

        thresh_frame_to_show = ResizeAndDisplayFrame("Bin", thresh_frame,
                                                     (org_w, 100))

        cv2.imwrite(os.path.join(frames_dir, f"{frame_count}.bmp"),
                    thresh_frame_to_show)

        if not recording and 255 in thresh_frame:
            recording = True
            spikeArr = []
            timeArr = []
            start_time = time.time()

        if recording and not 255 in thresh_frame:
            recording = False
            # print('Spikes:', spikeArr)
            # print('Times:', timeArr)
            # HERE WE SHOULD SEND spikeArr and timeArr TO THE NETWORK
            runNet(network, output, generator, spikeArr, timeArr*second)

        if recording:
            new_spikes = np.where(thresh_frame == 255)[1]
            new_times = np.ones(new_spikes.shape) * time.time() - start_time
            spikeArr.extend(new_spikes.tolist())
            timeArr.extend(new_times.tolist())
            # print('New Spikes:', new_spikes)
            # print('New Times:', new_times)

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# After the loop release the cap object
vid.release()
out_org.release()

# Destroy all the windows
cv2.destroyAllWindows()

Right to left! (Right spikes :7, Left spikes :6)
Undecided! (Right spikes :20, Left spikes :20)
Left to right! (Right spikes :20, Left spikes :23)
Right to left! (Right spikes :27, Left spikes :26)
Undecided! (Right spikes :23, Left spikes :23)
Right to left! (Right spikes :56, Left spikes :51)
Undecided! (Right spikes :15, Left spikes :15)
Left to right! (Right spikes :70, Left spikes :73)
Right to left! (Right spikes :61, Left spikes :56)
Undecided! (Right spikes :6, Left spikes :6)
Left to right! (Right spikes :64, Left spikes :70)
Right to left! (Right spikes :84, Left spikes :81)
Undecided! (Right spikes :2, Left spikes :2)
Left to right! (Right spikes :93, Left spikes :98)
